# re:Invent 2025 Workshop
# AIM406 - Build agentic workflows with Small Language Models and SageMaker AI
# Lab 2 - Bedrock Agent Core with Qwen Integration - Complete Guide

This notebook provides a comprehensive end-to-end guide for deploying a Bedrock Agent Core with Qwen model integration for intelligent research reports.

## Overview
- **Setup Docker in SageMaker Studio** for local development
- **Create IAM roles** with proper permissions
- **Deploy Qwen-powered research agent** to Bedrock Agent Core
- **Test locally and remotely** with comprehensive examples
- **Clean up resources** when finished

## Prerequisites
- SageMaker Studio environment
- AWS CLI configured
- Python 3.8+ environment

## 1. Environment Setup and Docker Installation

First, we'll install Docker in SageMaker Studio for local development and testing. This follows the AWS guide for SageMaker Studio Local Mode Docker support.

In [ ]:
# Install Docker in SageMaker Studio
import subprocess
import sys
import os

def install_docker():
    """Install Docker in SageMaker Studio following AWS guidelines"""
    try:
        # Check if Docker is already installed
        result = subprocess.run(['docker', '--version'], capture_output=True, text=True)
        if result.returncode == 0:
            print(f"Docker already installed: {result.stdout.strip()}")
            return True
    except FileNotFoundError:
        pass

    print("Installing Docker...")

    # Install Docker using the official installation script
    commands = [
        "sudo yum update -y",
        "sudo yum install -y docker",
        "sudo service docker start",
        "sudo usermod -a -G docker $USER"
    ]

    for cmd in commands:
        print(f"Running: {cmd}")
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if result.returncode != 0:
            print(f"Error: {result.stderr}")
            return False
        print(f"Success: {result.stdout}")

    print("Docker installation completed!")
    print("Note: You may need to restart your kernel for group changes to take effect.")
    return True

# Install Docker
install_docker()

In [ ]:
import json
import sagemaker
import boto3

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts
region = sess._region_name  # region name of the current SageMaker Studio environment

sm_client = boto3.client("sagemaker")  # client to intreract with SageMaker
smr_client = boto3.client("sagemaker-runtime")  # client to intreract with SageMaker Endpoints

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")
print(f"sagemaker version: {sagemaker.__version__}")

You can use AWS CLI to find a name for the endpoint we deployed in Lab1. For more information about AWS CLI please refer to the [documentation](https://docs.aws.amazon.com/cli/).

As an alternative, you can also use `boto3` API (AWS SDK for Python) to get a list of available endpoints. See AWS SDK for Python [documentation](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) for details.

In this workshop we will use AWS CLI.

In [ ]:
!aws sagemaker list-endpoints --status-equals "InService"

Please copy the endpoint name from the cell above to the cell below

In [ ]:
ENDPOINT = "model-251024-1552"


## 2. Install Required Dependencies

Install all necessary Python packages for Bedrock Agent Core development.

In [ ]:
# Install required packages
!pip install --force-reinstall -U -r requirements.txt
!pip install -q langchain-core
!pip install -q boto3
!pip install -q requests
!pip install -q beautifulsoup4

print("All dependencies installed successfully!")

## 3. Create IAM Role with Proper Permissions

Create a comprehensive IAM role that includes all necessary permissions for Bedrock Agent Core, SageMaker endpoint access, and other AWS services.

**Please note that if you are running this notebook as part of AWS workshop the role `BedrockAgentCoreQwenRole` was already created as part of the workshop setup.**
You should see the following message: 
```
Role BedrockAgentCoreQwenRole already exists: arn:aws:iam::ACCOUNT_ID:role/BedrockAgentCoreQwenRole
```

If you run this notebook in your own account please make sure you have permissions to create an IAM role.

In [ ]:
import boto3
import json
import time
from datetime import datetime

def create_bedrock_agent_role(role_name="BedrockAgentCoreQwenRole"):
    """Create IAM role with comprehensive permissions for Bedrock Agent Core"""

    iam = boto3.client('iam')

    # Trust policy for Bedrock Agent Core
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": [
                        "bedrock-agentcore.amazonaws.com",
                        "codebuild.amazonaws.com"
                    ]
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    # Comprehensive permissions policy
    permissions_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "sagemaker:InvokeEndpoint",
                    "sagemaker:InvokeEndpointAsync",
                    "sagemaker:DescribeEndpoint",
                    "sagemaker:ListEndpoints"
                ],
                "Resource": "*"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeModel",
                    "bedrock:InvokeModelWithResponseStream"
                ],
                "Resource": "*"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "logs:CreateLogGroup",
                    "logs:CreateLogStream",
                    "logs:PutLogEvents",
                    "logs:DescribeLogGroups",
                    "logs:DescribeLogStreams"
                ],
                "Resource": "*"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "codebuild:CreateProject",
                    "codebuild:UpdateProject",
                    "codebuild:StartBuild",
                    "codebuild:BatchGetBuilds",
                    "codebuild:BatchGetProjects"
                ],
                "Resource": "*"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "ecr:GetAuthorizationToken",
                    "ecr:BatchCheckLayerAvailability",
                    "ecr:GetDownloadUrlForLayer",
                    "ecr:GetRepositoryPolicy",
                    "ecr:DescribeRepositories",
                    "ecr:ListImages",
                    "ecr:DescribeImages",
                    "ecr:BatchGetImage",
                    "ecr:GetLifecyclePolicy",
                    "ecr:GetLifecyclePolicyPreview",
                    "ecr:ListTagsForResource",
                    "ecr:DescribeImageScanFindings",
                    "ecr:InitiateLayerUpload",
                    "ecr:UploadLayerPart",
                    "ecr:CompleteLayerUpload",
                    "ecr:PutImage"
                ],
                "Resource": "*"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "s3:GetObject",
                    "s3:PutObject",
                    "s3:CreateBucket",
                    "s3:ListBucket"
                ],
                "Resource": "*"
            },
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock:InvokeAgent",
                    "bedrock-agent-runtime:InvokeAgent",
                    "bedrock-agent:GetAgent",
                    "bedrock-agent:ListAgents",
                    "bedrock-agent:PrepareAgent",
                    "bedrock-agent:CreateAgentAlias",
                    "bedrock-agent:GetAgentAlias"
                ],
                "Resource": "*"
            }
        ]
    }

    try:
        # Check if role already exists
        try:
            role = iam.get_role(RoleName=role_name)
            print(f"Role {role_name} already exists: {role['Role']['Arn']}")
            return role['Role']['Arn']
        except iam.exceptions.NoSuchEntityException:
            pass

        # Create the role
        print(f"Creating IAM role: {role_name}")
        role_response = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description="Comprehensive role for Bedrock Agent Core with Qwen integration"
        )

        role_arn = role_response['Role']['Arn']
        print(f"Role created: {role_arn}")

        # Create and attach the permissions policy
        policy_name = f"{role_name}Policy"
        print(f"Creating policy: {policy_name}")

        policy_response = iam.create_policy(
            PolicyName=policy_name,
            PolicyDocument=json.dumps(permissions_policy),
            Description="Comprehensive permissions for Bedrock Agent Core with Qwen"
        )

        policy_arn = policy_response['Policy']['Arn']
        print(f"Policy created: {policy_arn}")

        # Attach policy to role
        iam.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )

        # Also attach AWS managed policy for Bedrock
        iam.attach_role_policy(
            RoleName=role_name,
            PolicyArn="arn:aws:iam::aws:policy/AmazonBedrockFullAccess"
        )

        print(f"Policies attached to role {role_name}")
        print("Waiting for role to propagate...")
        time.sleep(10)  # Wait for IAM propagation

        return role_arn

    except Exception as e:
        print(f"Error creating role: {e}")
        return None

# Create the IAM role
role_arn = create_bedrock_agent_role()
print(f"\nIAM Role ARN: {role_arn}")

### Create Qwen Client

The QwenClient handles communication with the SageMaker endpoint for AI-powered analysis.

In [ ]:
# Create qwen_client.py
qwen_client_code = f'''import boto3
import json

class QwenClient:
    def __init__(self, endpoint_name):
        self.endpoint_name = endpoint_name
        self.runtime = boto3.client('sagemaker-runtime', region_name='{region}')

    def invoke(self, messages):
        # Convert messages to Qwen format
        prompt = ""
        for msg in messages:
            if hasattr(msg, 'content'):
                if type(msg).__name__ == "HumanMessage":
                    prompt += f"Human: {{msg.content}}\\n"
                elif type(msg).__name__ == "AIMessage":
                    prompt += f"Assistant: {{msg.content}}\\n"
                elif type(msg).__name__ == "ToolMessage":
                    prompt += f"Tool Result: {{msg.content}}\\n"

        prompt += "Assistant:"

        payload = {{
            "inputs": prompt,
            "parameters": {{
                "max_new_tokens": 2000,
                "temperature": 0.1,
                "do_sample": True
            }}
        }}

        try:
            response = self.runtime.invoke_endpoint(
                EndpointName=self.endpoint_name,
                ContentType='application/json',
                Body=json.dumps(payload)
            )

            result = json.loads(response['Body'].read().decode())

            # Handle different response formats
            if isinstance(result, dict) and 'generated_text' in result:
                ret = result['generated_text'].split("Assistant:")[-1].strip()
            elif isinstance(result, list) and len(result) > 0:
                ret = result[0]['generated_text'].split("Assistant:")[-1].strip()
            else:
                ret = "Analysis completed using Qwen model."

            ret = '\\n'.join(line for line in ret.split('\\n') if line.strip() != 'Assistant')
            return ret

        except Exception as e:
            error_msg = f"Qwen endpoint error: {{str(e)}} | Type: {{type(e).__name__}}"
            print(error_msg)

            # Check if it's a permissions issue
            if "AccessDenied" in str(e) or "UnauthorizedOperation" in str(e):
                return f"Analysis completed (permission denied: {{self.endpoint_name}})."
            elif "ValidationException" in str(e):
                return f"Analysis completed (endpoint not found: {{self.endpoint_name}})."
            elif "ModelError" in str(e):
                return f"Analysis completed (model error: {{self.endpoint_name}})."
            else:
                return f"Analysis completed (endpoint error: {{type(e).__name__}})."
'''

with open("qwen_client.py", "w") as f:
    f.write(qwen_client_code)

print("Created qwen_client.py")

### Create Main Application

The main.py file contains the Bedrock Agent Core application with Qwen integration for intelligent research reports.

In [ ]:
# Create main.py - the core Bedrock Agent Core application
main_code = f'''from agent.tools import internet_search
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.runnables import Runnable
from qwen_client import QwenClient
import json
import re

app = BedrockAgentCoreApp()

def parse_clean_report(response_text):
    """Extract and clean report from Bedrock Agent Core response"""
    lines = response_text.strip().split('\\n')

    for line in lines:
        if line.startswith('data: '):
            try:
                data = json.loads(line[6:])
                messages = data.get('messages', [])

                # Find the last assistant message with report
                for msg in reversed(messages):
                    if (msg.get('type') == 'assistant' and
                        msg.get('content') and
                        '# Comprehensive Research Report' in msg.get('content', '')):

                        content = msg['content']

                        # Remove emojis
                        content = re.sub(r'[🎯📚📊💡🔍]', '', content)

                        # Clean up extra spaces
                        content = re.sub(r'\\n\\s*\\n', '\\n\\n', content)
                        content = re.sub(r'  +', ' ', content)

                        return content.strip()
            except:
                continue

    return "No report found"

class SimpleQwenWrapper(Runnable):
    def __init__(self, endpoint_name):
        self.endpoint_name = endpoint_name
        self.qwen = QwenClient(endpoint_name)

    def bind_tools(self, tools):
        return self

    def invoke(self, input_data, config=None, **kwargs):
        messages = input_data if isinstance(input_data, list) else input_data.get("messages", [])

        if not messages:
            return AIMessage(content="I'll help with research.")

        # Check if we have search results to summarize
        search_results = None
        for msg in messages:
            if isinstance(msg, ToolMessage) and "results" in getattr(msg, 'content', ''):
                try:
                    search_results = json.loads(msg.content)
                    break
                except:
                    pass

        # If we have search results, use Qwen to generate summary
        if search_results and "results" in search_results:
            # Call Qwen endpoint for report generation
            qwen_response = self.qwen.invoke(messages)

            # Generate clean report without emojis
            results = search_results["results"]
            sources = []
            findings = []
            for result in results[:5]:
                sources.append(f"- [{{result.get('title', 'N/A')}}]({{result.get('url', 'N/A')}})")
                findings.append(f"**{{result.get('title', 'Source')}}**: {{result.get('content', 'No content available')}}")

            clean_report = f"""# Comprehensive Research Report

## Research Query
{{search_results.get('query', 'Research query')}}

## Sources Analyzed
{{chr(10).join(sources)}}

## Key Findings
{{chr(10).join(findings)}}

## Analysis Summary
{{qwen_response}}

## Research Methodology
- Conducted targeted internet search via Qwen endpoint
- Analyzed multiple authoritative sources
- Generated insights using Qwen model
- Provided source citations for verification

---
Report generated using Qwen model: {{self.endpoint_name}}"""

            return AIMessage(content=clean_report)

        # If no search results yet, trigger search
        user_question = ""
        for msg in messages:
            if type(msg).__name__ == "HumanMessage":
                user_question = getattr(msg, 'content', '')
                break

        if user_question:
            return AIMessage(
                content="Conducting comprehensive research using Qwen model...",
                tool_calls=[{{
                    "name": "internet_search",
                    "args": {{"query": user_question, "max_results": 5}},
                    "id": "search_1"
                }}]
            )

        return AIMessage(content="Please provide a research question.")

    async def ainvoke(self, input_data, config=None, **kwargs):
        return self.invoke(input_data, config, **kwargs)

# Create simple wrapper
LLM = SimpleQwenWrapper(endpoint_name="{ENDPOINT}")

def serialize_message(message):
    if isinstance(message, HumanMessage):
        return {{
            "type": "human",
            "content": message.content,
            "id": getattr(message, "id", None),
        }}
    elif isinstance(message, AIMessage):
        return {{
            "type": "assistant",
            "content": message.content,
            "id": getattr(message, "id", None),
            "tool_calls": getattr(message, "tool_calls", []),
        }}
    elif isinstance(message, ToolMessage):
        return {{
            "type": "tool",
            "content": message.content,
            "name": getattr(message, "name", "unknown"),
        }}
    else:
        return {{
            "type": "unknown",
            "content": str(message),
        }}

def serialize_chunk(chunk):
    serialized_chunk = {{}}
    if "messages" in chunk:
        serialized_chunk["messages"] = []
        for message in chunk["messages"]:
            serialized_chunk["messages"].append(serialize_message(message))
    return serialized_chunk

@app.entrypoint
async def simple_qwen_research(payload):
    """Simple research using Qwen wrapper"""
    user_input = payload.get("prompt")

    # Step 1: Trigger search
    search_response = LLM.invoke([HumanMessage(content=user_input)])
    yield serialize_chunk({{"messages": [HumanMessage(content=user_input), search_response]}})

    # Step 2: Execute search
    if hasattr(search_response, 'tool_calls') and search_response.tool_calls:
        search_call = search_response.tool_calls[0]
        search_result = internet_search(**search_call['args'])
        tool_msg = ToolMessage(content=json.dumps(search_result), name="internet_search", tool_call_id=search_call['id'])
        yield serialize_chunk({{"messages": [HumanMessage(content=user_input), search_response, tool_msg]}})

        # Step 3: Generate summary using Qwen
        final_response = LLM.invoke([HumanMessage(content=user_input), search_response, tool_msg])
        yield serialize_chunk({{"messages": [HumanMessage(content=user_input), search_response, tool_msg, final_response]}})

if __name__ == "__main__":
    app.run()
'''

with open("main.py", "w") as f:
    f.write(main_code)

print("Created main.py - Bedrock Agent Core application")

### Create Local Testing Script

The test_working.py script allows you to test the Qwen integration locally before deploying to Bedrock Agent Core.

In [ ]:
# Create test_working.py for local testing
test_code = f'''from agent.tools import internet_search
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.runnables import Runnable
from qwen_client import QwenClient
import json
import asyncio

class SimpleQwenWrapper(Runnable):
    def __init__(self, endpoint_name):
        self.endpoint_name = endpoint_name
        self.qwen = QwenClient(endpoint_name)

    def bind_tools(self, tools):
        return self

    def invoke(self, input_data, config=None, **kwargs):
        messages = input_data if isinstance(input_data, list) else input_data.get("messages", [])

        if not messages:
            return AIMessage(content="I'll help with research.")

        # Check if we have search results to summarize
        search_results = None
        for msg in messages:
            if isinstance(msg, ToolMessage) and "results" in getattr(msg, 'content', ''):
                try:
                    search_results = json.loads(msg.content)
                    break
                except:
                    pass

        # If we have search results, use Qwen to generate summary
        if search_results and "results" in search_results:
            # Call Qwen endpoint for report generation
            qwen_response = self.qwen.invoke(messages)

            # Generate clean report without emojis
            results = search_results["results"]
            sources = []
            findings = []
            for result in results[:5]:
                sources.append(f"- [{{result.get('title', 'N/A')}}]({{result.get('url', 'N/A')}})")
                findings.append(f"**{{result.get('title', 'Source')}}**: {{result.get('content', 'No content available')}}")

            clean_report = f"""# Comprehensive Research Report

## Research Query
{{search_results.get('query', 'Research query')}}

## Sources Analyzed
{{chr(10).join(sources)}}

## Key Findings
{{chr(10).join(findings)}}

## Analysis Summary
{{qwen_response}}

## Research Methodology
- Conducted targeted internet search via Qwen endpoint
- Analyzed multiple authoritative sources
- Generated insights using Qwen model
- Provided source citations for verification

---
Report generated using Qwen model: {{self.endpoint_name}}"""

            return AIMessage(content=clean_report)

        # If no search results yet, trigger search
        user_question = ""
        for msg in messages:
            if type(msg).__name__ == "HumanMessage":
                user_question = getattr(msg, 'content', '')
                break

        if user_question:
            return AIMessage(
                content="Conducting comprehensive research using Qwen model...",
                tool_calls=[{{
                    "name": "internet_search",
                    "args": {{"query": user_question, "max_results": 5}},
                    "id": "search_1"
                }}]
            )

        return AIMessage(content="Please provide a research question.")

    async def ainvoke(self, input_data, config=None, **kwargs):
        return self.invoke(input_data, config, **kwargs)

# Create simple wrapper
LLM = SimpleQwenWrapper(endpoint_name="{ENDPOINT}")

async def simple_qwen_research(payload):
    """Simple research without deep agents - local version"""
    user_input = payload.get("prompt")

    # Step 1: Trigger search
    search_response = LLM.invoke([HumanMessage(content=user_input)])
    print("Step 1 - Search trigger:")
    print(f"Content: {{search_response.content}}")

    # Step 2: Execute search
    if hasattr(search_response, 'tool_calls') and search_response.tool_calls:
        search_call = search_response.tool_calls[0]
        search_result = internet_search(**search_call['args'])
        tool_msg = ToolMessage(content=json.dumps(search_result), name="internet_search", tool_call_id=search_call['id'])
        print("\\nStep 2 - Tool execution completed")

        # Step 3: Generate summary
        final_response = LLM.invoke([HumanMessage(content=user_input), search_response, tool_msg])
        print("\\nStep 3 - Final report:")
        print(final_response.content)

# Test locally
if __name__ == "__main__":
    query = "What is the latest in Quantum Computing today?"
    payload = {{"prompt": query}}
    asyncio.run(simple_qwen_research(payload))
'''

with open("test_working.py", "w") as f:
    f.write(test_code)

print("Created test_working.py - Local testing script")

## 5. Run Local Testing

Before deploying to Bedrock Agent Core, let's test the Qwen integration locally to ensure everything works correctly.

In [ ]:
# Run the local test
print("Running local test of Qwen integration...")
print("Note: This requires a deployed Qwen endpoint. Update the endpoint name in the code if needed.")
print("=" * 80)

In [ ]:
!python3 test_working.py

## 6. Deploy to Bedrock Agent Core

Now we'll deploy our Qwen-powered research agent to Bedrock Agent Core using the comprehensive IAM role we created.

In [ ]:
# Create deployment script
deploy_code = f'''from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session

agent_name = "qwen_research_agent"
boto_session = Session()
region = "{region}"

agentcore_runtime = Runtime()

# Configure the agent
print("Configuring Bedrock Agent Core...")
response = agentcore_runtime.configure(
    entrypoint="main.py",
    auto_create_execution_role=False,
    execution_role="{role_arn}",  # Use the role we created
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name,
)

print("Configuration completed. Starting deployment...")

# Deploy using CodeBuild (recommended for SageMaker Studio)
launch_result = agentcore_runtime.launch(use_codebuild=True)

agent_arn = launch_result.agent_arn
print(f"\\n🎉 Deployment successful!")
print(f"Agent ARN: {{agent_arn}}")
print(f"\\nYou can now test your agent using the ARN above.")
'''

with open("deploy.py", "w") as f:
    f.write(deploy_code)

print("Created deploy.py")
print("\nStarting deployment to Bedrock Agent Core...")
print("This may take several minutes...")

# Execute deployment
exec(open('deploy.py').read())

## 7. Test the Deployed Agent

Create a test client to verify that the deployed agent works correctly with the Qwen integration.

In [ ]:
# Create test client for the deployed agent
test_client_code = f'''import boto3
import json
from botocore.config import Config
import re

def parse_deployment_response(response_text):
    """Parse and format the final report in clean report style"""
    lines = response_text.strip().split('\\n')

    for line in lines:
        if line.startswith('data: '):
            try:
                data = json.loads(line[6:])
                messages = data.get('messages', [])

                # Find the last assistant message with the report
                for msg in reversed(messages):
                    if (msg.get('type') == 'assistant' and
                        msg.get('content') and
                        '# Comprehensive Research Report' in msg.get('content', '')):

                        content = msg['content']

                        # Handle escaped newlines from JSON
                        content = content.replace('\\\\n', '\\n')

                        # Format as clean report
                        return format_clean_report(content)
            except Exception as e:
                continue

    return "No report found"

def format_clean_report(content):
    """Format markdown content as clean report style"""

    # Remove markdown headers and replace with clean formatting
    content = re.sub(r'^# (.+)$', r'\\1', content, flags=re.MULTILINE)
    content = re.sub(r'^## (.+)$', r'\\n\\1:\\n' + '='*50, content, flags=re.MULTILINE)

    # Clean up markdown links - keep just the text
    content = re.sub(r'\\[([^\\]]+)\\]\\([^)]+\\)', r'\\1', content)

    # Format bold text
    content = re.sub(r'\\*\\*([^*]+)\\*\\*:', r'\\1:', content)

    # Clean up extra newlines and spaces
    content = re.sub(r'\\n\\s*\\n\\s*\\n', '\\n\\n', content)
    content = re.sub(r'  +', ' ', content)

    # Remove unicode characters
    content = re.sub(r'[\\u200b\\u200c\\u200d]', '', content)

    return content.strip()

def test_deployed_agent(agent_arn, query="What is the latest in Quantum Computing today?"):
    """Test the deployed Bedrock Agent Core"""

    config = Config(read_timeout=300, connect_timeout=60)
    client = boto3.client('bedrock-agentcore', region_name='{region}', config=config)

    payload = json.dumps({{"prompt": query}})

    try:
        print(f"Testing deployed agent with query: {{query}}")
        print("=" * 60)

        response = client.invoke_agent_runtime(
            agentRuntimeArn=agent_arn,
            runtimeSessionId='test-session-12345678901234567890123456789012345',
            payload=payload,
            qualifier="DEFAULT"
        )

        # Read response
        response_body = response['response'].read().decode('utf-8')

        # Parse and format the response
        formatted_report = parse_deployment_response(response_body)

        print("FORMATTED RESEARCH REPORT:")
        print("=" * 60)
        print(formatted_report)

        return formatted_report

    except Exception as e:
        print(f"Error testing agent: {{e}}")
        return None
'''

with open("test_client.py", "w") as f:
    f.write(test_client_code)

print("Created test_client.py")

# Note: You'll need to update the agent_arn from the deployment output
print("\nTo test your deployed agent, update the agent_arn in the next cell and run it.")

In [ ]:
# Test the deployed agent
# Replace this with your actual agent ARN from the deployment output
AGENT_ARN = "arn:aws:bedrock-agentcore:us-east-1:YOUR_ACCOUNT:runtime/YOUR_AGENT_ID"

# Load and execute the test
exec(open('test_client.py').read())

# Run the test
if AGENT_ARN != "arn:aws:bedrock-agentcore:us-east-1:YOUR_ACCOUNT:runtime/YOUR_AGENT_ID":
    result = test_deployed_agent(AGENT_ARN)
else:
    print("Please update AGENT_ARN with your actual agent ARN from the deployment output above.")

## 8. Response Parser Utility

Create a utility function to parse and clean responses from the Bedrock Agent Core for use in applications.

In [ ]:
# Create response parser utility
parser_code = '''import json
import re

def parse_bedrock_response(response_text):
    """Parse and clean Bedrock Agent Core streaming response"""
    lines = response_text.strip().split('\\n')

    for line in lines:
        if line.startswith('data: '):
            try:
                data = json.loads(line[6:])
                messages = data.get('messages', [])

                # Find the final report
                for msg in reversed(messages):
                    if (msg.get('type') == 'assistant' and
                        msg.get('content') and
                        '# Comprehensive Research Report' in msg.get('content', '')):

                        content = msg['content']
                        content = content.replace('\\\\n', '\\n')

                        # Clean formatting
                        content = re.sub(r'^# (.+)$', r'\\1', content, flags=re.MULTILINE)
                        content = re.sub(r'^## (.+)$', r'\\n\\1:\\n' + '='*50, content, flags=re.MULTILINE)
                        content = re.sub(r'\\[([^\\]]+)\\]\\([^)]+\\)', r'\\1', content)
                        content = re.sub(r'\\*\\*([^*]+)\\*\\*:', r'\\1:', content)
                        content = re.sub(r'\\n\\s*\\n\\s*\\n', '\\n\\n', content)
                        content = re.sub(r'  +', ' ', content)

                        return content.strip()
            except:
                continue

    return "No report found"

# Example usage
def example_usage():
    """Example of how to use the parser"""
    sample_response = '''data: {"messages": [{"type": "assistant", "content": "# Comprehensive Research Report\\n\\n## Research Query\\nSample query\\n\\n## Analysis Summary\\nSample analysis"}]}
    '''

    parsed = parse_bedrock_response(sample_response)
    print("Parsed Response:")
    print(parsed)

if __name__ == "__main__":
    example_usage()
'''

with open("response_parser.py", "w") as f:
    f.write(parser_code)

print("Created response_parser.py - Utility for parsing Bedrock responses")

# Test the parser
exec(open('response_parser.py').read())

## 9. Cleanup Resources

When you're finished with the project, clean up AWS resources to avoid unnecessary charges.

In [ ]:
# Create cleanup script
cleanup_code = '''import boto3
import json
from botocore.exceptions import ClientError

def cleanup_resources(agent_name="qwen_research_agent", role_name="BedrockAgentCoreQwenRole"):
    """Clean up AWS resources created during this tutorial"""

    print("Starting cleanup of AWS resources...")

    # Initialize clients
    bedrock_client = boto3.client('bedrock-agentcore', region_name='us-east-1')
    iam_client = boto3.client('iam')
    ecr_client = boto3.client('ecr', region_name='us-east-1')

    # 1. Delete Bedrock Agent Core (if you have the ARN)
    print("\\n1. Bedrock Agent Core:")
    print("   Note: You may need to manually delete the agent from the AWS console")
    print("   or use the specific agent ARN with the delete API.")

    # 2. Clean up ECR repository
    print("\\n2. ECR Repository:")
    try:
        repo_name = f"bedrock-agentcore-{agent_name}"
        ecr_client.delete_repository(
            repositoryName=repo_name,
            force=True
        )
        print(f"   ✅ Deleted ECR repository: {repo_name}")
    except ClientError as e:
        if e.response['Error']['Code'] == 'RepositoryNotFoundException':
            print(f"   ℹ️  ECR repository not found (may already be deleted)")
        else:
            print(f"   ❌ Error deleting ECR repository: {e}")

    # 3. Clean up IAM role and policies
    print("\\n3. IAM Resources:")
    try:
        # List attached policies
        attached_policies = iam_client.list_attached_role_policies(RoleName=role_name)

        # Detach all policies
        for policy in attached_policies['AttachedPolicies']:
            iam_client.detach_role_policy(
                RoleName=role_name,
                PolicyArn=policy['PolicyArn']
            )
            print(f"   ✅ Detached policy: {policy['PolicyName']}")

            # Delete custom policies (not AWS managed)
            if not policy['PolicyArn'].startswith('arn:aws:iam::aws:policy/'):
                try:
                    iam_client.delete_policy(PolicyArn=policy['PolicyArn'])
                    print(f"   ✅ Deleted custom policy: {policy['PolicyName']}")
                except ClientError as e:
                    print(f"   ❌ Error deleting policy {policy['PolicyName']}: {e}")

        # Delete the role
        iam_client.delete_role(RoleName=role_name)
        print(f"   ✅ Deleted IAM role: {role_name}")

    except ClientError as e:
        if e.response['Error']['Code'] == 'NoSuchEntity':
            print(f"   ℹ️  IAM role not found (may already be deleted)")
        else:
            print(f"   ❌ Error cleaning up IAM resources: {e}")

    print("\\n🧹 Cleanup completed!")
    print("\\nNote: Some resources may need to be manually deleted from the AWS console:")
    print("- Bedrock Agent Core agents")
    print("- CloudWatch log groups")
    print("- Any SageMaker endpoints you created")

# Uncomment the line below to run cleanup
# cleanup_resources()
print("Cleanup script created. Uncomment the last line to run cleanup when ready.")
'''

with open("cleanup.py", "w") as f:
    f.write(cleanup_code)

print("Created cleanup.py")
print("\n⚠️  IMPORTANT: Run cleanup.py when you're finished to avoid unnecessary AWS charges.")

## Summary

This notebook provided a complete end-to-end guide for:

### ✅ What We Accomplished
1. **Docker Setup** - Installed Docker in SageMaker Studio for local development
2. **IAM Configuration** - Created comprehensive IAM role with proper permissions
3. **Project Structure** - Built complete Bedrock Agent Core project with Qwen integration
4. **Local Testing** - Verified functionality before deployment
5. **Cloud Deployment** - Deployed to Bedrock Agent Core using CodeBuild
6. **Remote Testing** - Validated deployed agent functionality
7. **Utilities** - Created response parsing and cleanup tools

### 🔧 Key Components Created
- **main.py** - Bedrock Agent Core application with Qwen integration
- **qwen_client.py** - SageMaker endpoint client for AI analysis
- **agent/tools.py** - Internet search functionality
- **test_working.py** - Local testing script
- **deploy.py** - Deployment automation
- **test_client.py** - Remote testing client
- **response_parser.py** - Response formatting utility
- **cleanup.py** - Resource cleanup automation

### 🚀 Next Steps
1. **Customize** the search tools and analysis prompts for your use case
2. **Integrate** the agent into your applications using the provided client code
3. **Monitor** performance and costs through AWS CloudWatch
4. **Scale** by adjusting the SageMaker endpoint configuration

### 🧹 Don't Forget
Run the cleanup script when finished to avoid unnecessary AWS charges!

```python
exec(open('cleanup.py').read())
cleanup_resources()  # Uncomment this line in cleanup.py
```